---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

<!-- #region id="89902a6f" -->
# Table des matières
1. [Méthodes de régularisation fréquemment utilisées](#Méthodes-de-régularisation-fréquemment-utilisées)
  1. [Régularisation en ajoutant des contraintes à la méthode des moindres carrés](#Régularisation-en-ajoutant-des-contraintes-à-la-méthode-des-moindres-carrés)
1. [Préparation des données et initialisations](#Préparation-des-données-et-initialisations)
1. [Exemples de régression à un polynôme de degré donné](#Exemples-de-régression-à-un-polynôme-de-degré-donné)
1. [Exemples de régression à des polynômes de degrés variés](#Exemples-de-régression-à-des-polynômes-de-degrés-variés)
1. [Affichage des courbes recalées avec les deux méthodes de régression](#Affichage-des-courbes-recalées-avec-les-deux-méthodes-de-régression)
1. [Exercices](#exercices)

# Attention!
Ne lancez pas l'exécution automatique du notebook en entier en cliquant sur le bouton **Tout exécuter**. L'exécution serait interrompue, car certaines cellules exigent une entrée de votre part!

Il faut simplement exécuter le notebook, une cellule à la fois, et entrer quelques lignes de code lorsque demandées. Il est inutile de sauter ces cellules pour aller aux suivantes car celles-ci ont justement besoin de votre input!

Importons d'abord les librairies nécessaires.
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model, metrics
from sklearn.linear_model import (
    ElasticNet,
    ElasticNetCV,
    Lasso,
    LassoCV,
    LinearRegression,
    Ridge,
    RidgeCV,
)
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (
    GridSearchCV,
    ShuffleSplit,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

sns.set(color_codes=True)


seed = 42
np.random.seed(seed)



<!-- #region id="8c18324e" -->
<p>&nbsp;</p>
<div align="center">
    <img src= "../images/regularization-illustration.png"  width="350" />
    <div>
    <font size="1.5">Image Source: https://commons.wikimedia.org/wiki/File:Regularization.svg</font>
    </div>
</div>
<p>&nbsp;</p>
<!-- #endregion -->

<!-- #region id="2081e838" -->
On a vu dans le module sur le sous-apprentissage et le surapprentissage que les méthodes de régression
et de classification sont sensibles au bruit dans les données. Il existe plusieurs méthodes permettant
de réduire l'effet du bruit dans les analyses. Elles sont connues sous le nom de régularisation,
car elles ont pour tâche de régulariser, c'est-à-dire, de mieux contrôler l'analyse des données.

Comme pour le module précédent, nous allons nous concentrer sur les méthodes utilisées en régression
et allons utiliser des modèles polynomiaux pour recaler des données expérimentales. Ils sont faciles à utiliser et les
résultats de la régularisation sont faciles à interpréter.
<!-- #endregion -->

<!-- #region id="fa794ab8" -->
# <a id=Méthodes-de-régularisation-fréquemment-utilisées>Méthodes de régularisation fréquemment utilisées</a>
<!-- #endregion -->

<!-- #region id="7e820681" -->
Il existe plusieurs méthodes de régularisation en apprentissage automatique. Les plus connues sont probablement les
suivantes:

- l'augmentation du nombre de données,
- la réduction de la complexité d'un modèle,
- l'ajout de contraintes à la méthode des moindres carrés.


La première méthode est la plus intuitive et la plus facile à essayer. Il suffit de faire l'acquisition de plus
de données lorsque le budget d'un projet le permet. Si cela n'est pas possible, on peut faire de l'augmentation
de données à partir de données existantes. C'est une technique largement utilisée en apprentissage profond.

La seconde méthode consiste à simplifier le modèle à recaler sur les données. On peut prendre par exemple un polynôme
de faible degré qui varie lentement, sans les grandes oscillations caractérisant ceux de degré élevé. La simplicité est
une bonne chose, mais un modèle trop simple ne tire pas parti de nos données.

La troisième méthode consiste à ajouter des contraintes à la méthode des moindres carrés. Nous allons
nous concentrer sur cette approche dans ce module.
<!-- #endregion -->

<!-- #region id="7fa960e5" -->
## <a id=Régularisation-en-ajoutant-des-contraintes-à-la-méthode-des-moindres-carrés>Régularisation en ajoutant des contraintes à la méthode des moindres carrés</a>
<!-- #endregion -->

<!-- #region id="aa6bd3ba" -->
Supposons que l'on veut estimer les paramètres $a_j$ du modèle 1-D suivant

$$y = h(x|\Theta)$$

où $\Theta$ est l'ensemble des paramètres $a_j$.
On veut recaler le modèle à des données expérimentales $(x^{(i)}, y^{(i)})$
sans erreurs de mesures. L'indice $i$ représente le numéro d'une donnée tel que $1\le i \le n$, où $n$ est le nombre de données. On a vu dans un module précédent qu’on peut estimer ces
coefficients avec la méthode des moindres carrés qui minimise la
métrique de l'erreur quadratique moyenne (MSE) définie comme suit

$$\text{MSE} = \frac{1}{n}\sum_{i=1} ^{n}(y^{(i)}-h(x^{(i)}|\Theta))^2$$

La méthode des moindres carrés est particulièrement utile pour estimer les paramètres $a_j$ des
modèles linéaires dans les paramètres comme les suivants par exemple:

Modèle multilinéaire $N$-D:
$$y = a_0 + a_1 x_1 + \cdots + a_{N} x_{N}.$$

Modèle polynomial $1$-D:
$$y = a_0 + a_1 x + a_2 x^2 + \cdots + a_{N} x^{N}.$$

Un problème classique avec la méthode des moindres carrés se produit lorsqu'un modèle contient
trop de coefficients $a_j$; l'ajustement de chacun tend à reproduire la distribution
des données bruitées plutôt que l'aspect général de la courbe sous-jacente. De plus,
les valeurs des coefficients deviennent arbitrairement grandes en valeurs absolues ce qui
limite l'interprétation des résultats.

La régularisation ajoute une contrainte dans le problème des moindres carrés. On définit une
nouvelle fonction à minimiser, la fonction de perte (*Loss*) qui remplace la MSE.
En voici quelques exemples:

$$\begin{align}
L_{\text{Standard}} &= \text{MSE} \\
L_{\text{Lasso}} &= \text{MSE} + \lambda \|\Theta\|^1 \\
L_{\text{Ridge}} &= \text{MSE} + \lambda \|\Theta\|^2 \\
L_{\text{Elastic Net}} &= \text{MSE} + \lambda_1 \|\Theta\|^1 + \lambda_2 \|\Theta\|^2 \\
\end{align}$$

avec les normes $\|\Theta\|^1$ et $\|\Theta\|^2$ qui sont calculées à partir des coefficients des modèles
$$\|\Theta\|^1 = \sum_{j=0} ^{N}|a_j |$$
$$\|\Theta\|^2 = \sum_{j=0} ^{N}a_j^2.$$

Les paramètres $\lambda$ sont positifs. On voit que la minimisation de la fonction de
perte $L$ implique à la fois la minimisation de la MSE et des valeurs des coefficients $a_j$.

Dans ce qui suit, nous n'allons discuter que de la méthode Ridge par souci de simplicité. Notons
toutefois les points suivants:


- la méthode Ridge est utilisée lorsque toutes les variables $x_j$ et leurs interactions $x_j x_k$ sont considérées importantes,

- la méthode Lasso est utilisée lorsque l'on a beaucoup de variables et que l'on se doute que plusieurs d'entre elles.
ne sont pas importantes,

- 
dans l'hésitation, on choisit la méthode Elastic Net qui est une combinaison des deux.


Les trois méthodes mettent à zéro (Lasso) ou près de zéro (Ridge et Elastic Net) les coefficients $a_j$ des variables
$x_j$ non importantes. Cela facilite l'interprétation des modèles en régression.
<!-- #endregion -->

<!-- #region id="fdc341bd" -->
# <a id=Préparation-des-données-et-initialisations>Préparation des données et initialisations</a>
<!-- #endregion -->

<!-- #region id="4d79105e" -->
Définissons d'abord quelques fonctions qui seront utilisées à plusieurs reprises. Par la suite, nous allons générer le vrai 
signal idéal que l'on va utiliser pour générer les données. Ce signal sera généré selon la fonction sinusoïdale suivante

$$y = 100\sin(10x).$$
<!-- #endregion -->



In [ ]:
# Génération du signal idéal
def modele(x):
    y = 100 * np.sin(10 * x)
    return y


# Génération d'échantillons du signal idéal et ajout de bruit gaussien.
# Les valeurs de x sont réparties aléatoirement sur la plage des valeurs
# disponibles.
def genere_signal_bruité(N, sigma):
    x = np.random.uniform(x_min, x_max, N)
    y = modele(x) + np.random.normal(0.0, sigma, N)

    df = pd.DataFrame({"X": x, "Y": y})
    return df


In [ ]:
# Génération du signal idéal pour des valeurs de x réparties uniformément

x_min = 0
x_max = 1

xx = np.linspace(x_min, x_max, 100)[:, np.newaxis]
yy = modele(xx)


In [ ]:
# Génération du signal expérimental (c.-à-d. bruité) pour des valeurs de x réparties aléatoirement

sigma = 60  # Niveau de bruit
N = 100  # Nombre de points du signal
df = genere_signal_bruité(N, sigma)



<!-- #region id="5d0aafc2" -->
La prochaine cellule définit une fonction qui effectue 100 fois les opérations suivantes pour un modèle de régression donné:

- séparation des données en ensembles d'entraînement et de test,
- entraînement du modèle avec l'ensemble d'entraînement, 
- prédiction de la réponse y avec les données d'entraînement,
- calcul des métriques $R^2$ et MSE,
- prédiction de la réponse y avec les données de test,
- calcul des métriques $R^2$ et MSE.

Les valeurs moyennes des métriques sont ensuite calculées pour les deux ensembles. Le nombre de répétitions
permet d'obtenir de meilleures statistiques.
<!-- #endregion -->



In [ ]:
def calculeMetriques(models, data, iterations=100):
    X = data["X"].to_numpy().reshape(-1, 1)
    Y = data["Y"].to_numpy().reshape(-1, 1)

    results = {}
    for i in models:
        r2_train = []
        r2_test = []
        mse_train = []
        mse_test = []

        for j in range(iterations):
            # Séparation des données en ensembles d'entraînement et de test
            X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

            # ------- Ensemble d'entraînement------
            # Entraînement du modèle
            models[i].fit(X_train, y_train)

            # Prédiction des valeurs de y
            y_pred = models[i].predict(X_train)

            # Calcul des métriques R2 et MSE
            r2_train.append(metrics.r2_score(y_train, y_pred))
            mse_train.append(metrics.mean_squared_error(y_train, y_pred))

            # ------- Ensemble de test------
            # Prédiction des valeurs de y
            y_pred = models[i].predict(X_test)

            # Calcul des métriques R2 et MSE
            r2_test.append(metrics.r2_score(y_test, y_pred))
            mse_test.append(metrics.mean_squared_error(y_test, y_pred))

        # Calcul des valeurs moyenne des métriques R2 et MSE pour les
        # deux ensembles.
        results[i] = [
            np.mean(r2_train),
            np.mean(r2_test),
            np.mean(mse_train),
            np.mean(mse_test),
        ]
        df = pd.DataFrame(
            results, index=["r2_train", "r2_test", "mse_train", "mse_test"]
        )

    return df



<!-- #region id="c98c07e8" -->
# <a id=Exemples-de-régression-à-un-polynôme-de-degré-donné>Exemples de régression à un polynôme de degré donné</a>
<!-- #endregion -->

<!-- #region id="36cda29b" -->
Nous allons comparer les performances des deux méthodes de régression, soit la méthode standard et
la méthode Ridge. Alors que la première ajuste les coefficients $a_i$ des polynômes pour minimiser la MSE,
la méthode Ridge minimise en même temps les valeurs des mêmes paramètres. On a ainsi

$$\begin{align}
L_{\text{Standard}} &= \text{MSE} \\
L_{\text{Ridge}} &= \text{MSE} + \lambda \sum_{j=0}^{N}a_j^2 \\
\end{align}$$

Lorsque la méthode standard est utilisée, les coefficients $a_j$ tendent à devenir très positifs ou très négatifs lorsqu'on essaie de recaler un polynôme de degré élevé à des données expérimentales. C'est le phénomène du surapprentissage. La régularisation empêche les valeurs des coefficients d'exploser.

Dans ce qui suit, pour simplifier l'analyse des données, nous allons concevoir des pipelines implémentant
toutes les étapes menant à la régression des données. Les deux pipelines sont très similaires
à une exception près; la méthode de recalage (*fit*).

Chaque pipeline comprend la création des vecteurs de caractéristiques

$$X = \begin{pmatrix} \bf{1} & \bf{x} & \bf{x^2} & \cdots & \bf{x^N}  \end{pmatrix}$$

où $N$ est le degré d'un polynôme puis leur normalisation, et enfin la régression.

À titre d'exemple, nous allons effectuer une régression avec des polynômes de degré 12.

<!-- #endregion -->



In [ ]:
# Pipeline pour la régression standard des moindres carrés
pipe1 = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=12)),
        ("scale", StandardScaler()),
        ("fit", linear_model.LinearRegression()),
    ]
)

# Pipeline pour la régression avec la méthode Ridge
pipe2 = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=12)),
        ("scale", StandardScaler()),
        ("fit", linear_model.Ridge()),
    ]
)

models1 = {"Standard": pipe1, "Ridge": pipe2}



<!-- #region id="c4bf6032" -->
Régression des données avec les deux pipelines et affichage des métriques $R^2$ et MSE pour les ensembles d'entraînement et de test.
<!-- #endregion -->



In [ ]:
calculeMetriques(models1, df)



<!-- #region id="285d6a2b" -->
Comme attendu, on voit que pour chaque méthode, les métriques de performances $R^2$ et MSE sont meilleures
avec l'ensemble d'entraînement qu'avec l'ensemble de test. Si l'on compare maintenant les deux méthodes
entre elles, on voit que la méthode Ridge performe nettement moins bien que la méthode standard.
La raison en est que l'algorithme de la classe `Ridge` de Scikit-learn utilise une valeur par défaut
(et non optimale dans ce cas-ci) pour le paramètre $\lambda$.

Nous allons reprendre l'exemple en spécifiant, cette fois-ci, une plage de valeurs possibles de $\lambda$ et utiliser
l'optimisation sur grille de paramètres avec la fonction GridSearchCV. Cela veut dire que la régression précédente
avec la méthode Ridge sera effectuée pour chaque valeur de $\lambda$ et que le meilleur modèle sera utilisé pour
calculer les métriques finales de performances $R^2$ et MSE.

> À noter que la classe `Ridge` de Scikit-learn utilise un facteur $\alpha$ plutôt que $\lambda$, plus souvent mentionné dans la littérature.
<!-- #endregion -->



In [ ]:
ridge_params = [{"fit__alpha": np.logspace(-8, 0, num=9)}]
models2 = {
    "Standard": pipe1,
    "Ridge": GridSearchCV(pipe2, param_grid=ridge_params, refit=True),
}



<!-- #region id="4ab5d4a6" -->
Régression des données avec les deux pipelines et affichage des métriques $R^2$ et MSE pour les ensembles d'entraînement et de test
<!-- #endregion -->



In [ ]:
calculeMetriques(models2, df)



<!-- #region id="abfec00f" -->
Les résultats en test montrent que la méthode Ridge performe maintenant mieux que l'entraînement précédent, mais ne permet pas de mieux performer que la méthode standard pour les deux métriques MSE et $R^2$.
<!-- #endregion -->

<!-- #region id="cc4171c3" -->
# <a id=Exemples-de-régression-à-des-polynômes-de-degrés-variés>Exemples de régression à des polynômes de degrés variés</a>
<!-- #endregion -->

<!-- #region id="a0552174" -->
Nous allons reprendre l'exemple précédent, mais en faisant varier maintenant le degré du polynôme de
régression. Ainsi, pour chaque polynôme, la valeur optimale du paramètre $\lambda$ sera ajustée.

Cela nous permettra de comparer les performances des deux méthodes de régression en fonction de la complexité du
modèle que l'on veut recaler aux données.

La fonction suivante permet de refaire, pour chaque degré de polynôme, les opérations précédentes.

<!-- #endregion -->



In [ ]:
def regression_modeles(df, degre):
    pipe1 = Pipeline(
        [
            ("poly", PolynomialFeatures(degree=degre)),
            ("scale", StandardScaler()),
            ("fit", linear_model.LinearRegression()),
        ]
    )

    pipe2 = Pipeline(
        [
            ("poly", PolynomialFeatures(degree=degre)),
            ("scale", StandardScaler()),
            ("fit", linear_model.Ridge()),
        ]
    )

    ridge_params = [{"fit__alpha": np.logspace(-8, 0, num=9)}]
    modeles = {
        "Standard": pipe1,
        "Ridge": GridSearchCV(pipe2, param_grid=ridge_params, refit=True),
    }

    return calculeMetriques(modeles, df)



<!-- #region id="824fd814" -->
Nous allons maintenant calculer les métriques de performances $R^2$ et MSE pour des polynômes
d'ordre variant entre 3 et 12.
<!-- #endregion -->



In [ ]:
stats1 = {}

degres = range(3, 13)

for i, degre in enumerate(degres):
    print(f"{i = }")
    stats1[i] = regression_modeles(df, degre)



<!-- #region id="dba57a83" -->
La fonction suivante permet d'afficher les métriques de performances $R^2$ et MSE en fonction
du degré des polynômes, soit la complexité du modèle de régression.
<!-- #endregion -->



In [ ]:
def affiche_resultats(stats, degres):
    fig, axes = plt.subplots(2, 1, figsize=(10, 10))

    # Extraction des valeurs de la métrique MSE
    mse_train_1 = []
    mse_test_1 = []
    mse_train_2 = []
    mse_test_2 = []

    for i, degre in enumerate(degres):
        mse_train_1.append(stats[i]["Standard"]["mse_train"])
        mse_test_1.append(stats[i]["Standard"]["mse_test"])
        mse_train_2.append(stats[i]["Ridge"]["mse_train"])
        mse_test_2.append(stats[i]["Ridge"]["mse_test"])

    # Affichage des courbes MSE versus degré
    axes[0].plot(degres, mse_train_1, "k-", label="Standard, Train")
    axes[0].plot(degres, mse_test_1, "k--", label="Standard, Test")
    axes[0].plot(degres, mse_train_2, "r-", label="Ridge, Train")
    axes[0].plot(degres, mse_test_2, "r--", label="Ridge, Test")

    axes[0].set_xlabel("Degré", fontsize=14)
    axes[0].set_ylabel("$MSE$", fontsize=14)
    axes[0].set_title(
        "Évolution de l'erreur quadratique moyenne en fonction du degré des polynômes",
        fontsize=14,
    )
    axes[0].legend()

    # Extraction des valeurs de la métrique R^2
    r2_train_1 = []
    r2_test_1 = []
    r2_train_2 = []
    r2_test_2 = []

    for i, degre in enumerate(degres):
        r2_train_1.append(stats[i]["Standard"]["r2_train"])
        r2_test_1.append(stats[i]["Standard"]["r2_test"])
        r2_train_2.append(stats[i]["Ridge"]["r2_train"])
        r2_test_2.append(stats[i]["Ridge"]["r2_test"])

    # Affichage des courbes R2 versus degré
    axes[1].plot(degres, r2_train_1, "k-", label="Standard, Train")
    axes[1].plot(degres, r2_test_1, "k--", label="Standard, Test")
    axes[1].plot(degres, r2_train_2, "r-", label="Ridge, Train")
    axes[1].plot(degres, r2_test_2, "r--", label="Ridge, Test")
    axes[1].set_ylim([0, 1])

    axes[1].set_xlabel("Degré", fontsize=14)
    axes[1].set_ylabel("$R^2$", fontsize=14)
    axes[1].set_title(
        "Évolution de la métrique du $R^2$ en fonction du degré des polynômes",
        fontsize=14,
    )
    axes[1].legend()
    plt.tight_layout()



<!-- #region id="d00c93af" -->
Affichage des métriques de MSE et du $R^2$ en fonction du degré des polynômes.
<!-- #endregion -->



In [ ]:
affiche_resultats(stats1, degres)



<!-- #region id="ec1a246a" -->
Considérons d'abord les résultats en entraînement; ils sont assez similaires pour les deux méthodes.

Les résultats en test sont plus intéressants. La régularisation n'affecte pas vraiment le sous-apprentissage.
Les résultats de $R^2$ et de MSE sont similaires pour les deux méthodes jusqu'au minimum de MSE, là
où se termine le sous-apprentissage.

Notons que maintenant la régularisation réduit et retarde l'apparition du surapprentissage. Alors que la MSE et le $R^2$ de la méthode standard se dégradent pour des degrés plus grands que $6$, les deux métriques Ridge restent plus ou moins constantes pour les modèles plus complexes!
<!-- #endregion -->

<!-- #region id="caca36c8" -->
Pourquoi est-ce que la régularisation n'affecte pas vraiment le sous-apprentissage?
<!-- #endregion -->

<!-- #region id="e837516d" -->
**Réponse: Les coefficients des polynômes de faibles degrés n'explosent jamais lors d'un recalage. La régularisation doit quand même les réduire un peu, en valeur absolue. Toutefois pas assez pour produire un changement dans les figures.**
<!-- #endregion -->

<!-- #region id="23e8a6ce" -->
# <a id=Affichage-des-courbes-recalées-avec-les-deux-méthodes-de-régression>Affichage des courbes recalées avec les deux méthodes de régression</a>
<!-- #endregion -->

<!-- #region id="86f83127" -->
Définissons d'abord une fonction d'affichage permettant de comparer les prédictions des modèles standard et Ridge avec les données expérimentales.
<!-- #endregion -->



In [ ]:
def plot_data(df, pipe1, pipe2, xx, yy):
    X = df["X"].to_numpy().reshape(-1, 1)
    Y = df["Y"].to_numpy().reshape(-1, 1)

    pipe1.fit(X, Y)
    y_pred_1 = pipe1.predict(xx)

    pipe2.fit(X, Y)
    y_pred_2 = pipe2.predict(xx)

    fig, ax = plt.subplots(1, 1, figsize=(15, 10))

    ax.scatter(X, Y, color="black", alpha=0.1)
    ax.plot(xx, yy, "k-", label="Vrai", lw=20, alpha=0.1)
    ax.plot(xx, y_pred_1, "r-", label="Standard")
    ax.plot(xx, y_pred_2, "g-", label="Ridge")
    ax.legend()



<!-- #region id="4a6b5472" -->
Comparons les résultats de chaque méthode pour le degré de polynôme optimal, c'est-à-dire, pour lequel la MSE est minimale.
Selon la figure précédente, le minimum de MSE de la méthode standard est atteint avec un polynôme de degré 6 alors celui de MSE avec la méthode de Ridge est atteint avec une valeur de 11.
Entraînons les modèles correspondants.
<!-- #endregion -->



In [ ]:
ridge_params = [{"fit__alpha": np.logspace(-8, 0, num=9)}]

pipe1 = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=6)),
        ("scale", StandardScaler()),
        ("fit", linear_model.LinearRegression()),
    ]
)

pipe = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=11)),
        ("scale", StandardScaler()),
        ("fit", linear_model.Ridge()),
    ]
)
ridge_params = [{"fit__alpha": np.logspace(-8, 0, num=9)}]
pipe2 = GridSearchCV(pipe, param_grid=ridge_params, refit=True)



<!-- #region id="b733bd14" -->
Affichage des prédictions des deux modèles lorsque les erreurs de régression sont minimales.
<!-- #endregion -->



In [ ]:
plot_data(df, pipe1, pipe2, xx, yy)



<!-- #region id="318c3cbc" -->
La figure montre que les prédictions des deux modèles reproduisent assez bien la distribution des
valeurs expérimentales ainsi que la courbe du modèle utilisé soit

$$y = 100\sin(10x)$$

Comparons maintenant les résultats pour des polynômes de degré 11, là où la méthode standard est proie au
surapprentissage, et où la méthode Ridge résiste très bien. Entraînons les modèles correspondants.

<!-- #endregion -->



In [ ]:
ridge_params = [{"fit__alpha": np.logspace(-8, 0, num=9)}]

pipe1 = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=11)),
        ("scale", StandardScaler()),
        ("fit", linear_model.LinearRegression()),
    ]
)

pipe = Pipeline(
    [
        ("poly", PolynomialFeatures(degree=11)),
        ("scale", StandardScaler()),
        ("fit", linear_model.Ridge()),
    ]
)
ridge_params = [{"fit__alpha": np.logspace(-8, 0, num=9)}]
pipe2 = GridSearchCV(pipe, param_grid=ridge_params, refit=True)



<!-- #region id="9ebff413" -->
Affichage des prédictions des deux modèles dans un régime de surapprentissage.
<!-- #endregion -->



In [ ]:
plot_data(df, pipe1, pipe2, xx, yy)



<!-- #region id="d67643f1" -->
La figure montre que la régression Ridge reproduit assez bien le signal original alors que la
régression standard présente des oscillations typiques du surapprentissage. La régularisation
contrôle bien les oscillations parasites!
<!-- #endregion -->

<!-- #region id="06679b5a" -->
# <a id=Exercices>Exercices</a>
<!-- #endregion -->

<!-- #region id="6772dab1" -->
Quel est l'effet de diminuer le nombre de données sur la régularisation?
Pour y répondre, faites ceci en réutilisant le code précédent:

- mettez le nombre de points du signal à 75,
- gardez le niveau de bruit à 60,
- réentrainez le pipeline précédent,
- affichez les résultats.
<!-- #endregion -->



In [ ]:
np.random.seed(seed)

sigma = 60  # Niveau de bruit
N = 75  # Nombre de points du signal

# Génération du signal expérimental (c.-à-d. bruité) pour des valeurs
# de x réparties aléatoirement.
df = genere_signal_bruité(N, sigma)

# Entraînement du pipeline précédent
pipe2 = GridSearchCV(pipe, param_grid=ridge_params, refit=True)

# Affichage des résultats
plot_data(df, pipe1, pipe2, xx, yy)



<!-- #region id="1ca1558f" -->
Que remarquez-vous?
<!-- #endregion -->

<!-- #region id="744ee49a" -->
**Réponse: la diminution du nombre de points augmente les oscillations de la
solution standard, sans régularisation. La solution Ridge est plus robuste au bruit
et tend à bien reproduire le signal original, non bruité.**
<!-- #endregion -->

<!-- #region id="d41909de" -->
Quel est l'effet d'augmenter le nombre de données sur la régularisation?
Pour y répondre, faites ceci en réutilisant le code précédent:

- mettez le nombre de points du signal à 1 000,
- gardez le niveau de bruit à 60,
- réentrainez le pipeline précédent,
- affichez les résultats.
<!-- #endregion -->



In [ ]:
np.random.seed(seed)

sigma = 60  # Niveau de bruit
N = 1000  # Nombre de points du signal

# Génération du signal expérimental (c.-à-d. bruité) pour des valeurs
# de x réparties aléatoirement.
df = genere_signal_bruité(N, sigma)

# Entraînement du pipeline précédent
pipe2 = GridSearchCV(pipe, param_grid=ridge_params, refit=True)

# Affichage des résultats
plot_data(df, pipe1, pipe2, xx, yy)



<!-- #region id="62924478" -->
Que remarquez-vous?
<!-- #endregion -->

<!-- #region id="5565199d" -->
**Réponse: l'augmentation du nombre de points diminue les oscillations
des deux méthodes; la régression Ridge devient inutile. Pourquoi? Parce que l'augmentation
du nombre de données est une autre méthode de régularisation telle que mentionnée au début du module.**
<!-- #endregion -->

<!-- #region id="9b4f060a" -->
Quel est l'effet de diminuer le niveau de bruit sur la régularisation?
Pour y répondre, faites ceci en réutilisant le code précédent:

- mettez le nombre de points du signal à 100,
- mettez le niveau de bruit à 10,
- réentrainez le pipeline précédent,
- affichez les résultats.
<!-- #endregion -->



In [ ]:
np.random.seed(seed)

sigma = 10  # Niveau de bruit
N = 100  # Nombre de points du signal

# Génération du signal expérimental (c.-à-d. bruité) pour des valeurs
# de x réparties aléatoirement.
df = genere_signal_bruité(N, sigma)

# Entraînement du pipeline précédent
pipe2 = GridSearchCV(pipe, param_grid=ridge_params, refit=True)

# Affichage des résultats
plot_data(df, pipe1, pipe2, xx, yy)



<!-- #region id="848573c4" -->
Que remarquez-vous?
<!-- #endregion -->

<!-- #region id="787f6b77" -->
**Réponse: la diminution du niveau de bruit réduit les oscillations des deux méthodes de régularisation. Pourquoi? Parce que le signal bruité n'est plus très complexe; il n'a plus besoin de polynômes de degré élevé pour reproduire la distribution des points.**
<!-- #endregion -->

<!-- #region id="ccd95f55" -->
Quel est l'effet d'augmenter le niveau de bruit sur la régularisation?
Pour y répondre, faites ceci en réutilisant le code précédent:

- mettez le nombre de points du signal à 100,
- mettez le niveau de bruit à 150,
- réentrainez le pipeline précédent,
- affichez les résultats.
<!-- #endregion -->



In [ ]:
np.random.seed(seed)

sigma = 150  # Niveau de bruit élevé
N = 100  # Nombre de points du signal

# Génération du signal expérimental (c.-à-d. bruité) pour des valeurs
# de x réparties aléatoirement.
df = genere_signal_bruité(N, sigma)

# Entraînement du pipeline précédent
pipe2 = GridSearchCV(pipe, param_grid=ridge_params, refit=True)

# Affichage des résultats
plot_data(df, pipe1, pipe2, xx, yy)



<!-- #region id="b0ece3d1" -->
Que remarquez-vous?
<!-- #endregion -->

<!-- #region id="37299444" -->
**Réponse: l'augmentation du niveau de bruit dégrade considérablement
les performances des deux méthodes de régularisation. Le signal bruité devient trop complexe
à reproduire. Néanmoins, la méthode Ridge donne les meilleurs résultats, mais de peu. On pourrait améliorer les résultats de la méthode Ridge en déterminant à nouveau l'ordre optimal du polynôme. Il ne serait plus de 11 comme dans la secton précédente.**
<!-- #endregion -->
